In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import random
from  tqdm import tqdm
import os

from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from pandas import Series, DataFrame
import datetime as dt

In [2]:
# 每一个事件有相应的id作为标识,调用一存储的json 文件

with open('event2id.json', 'r') as f:
    event2id = json.load(f)

In [3]:
# 同时生成id对应的event
def dict_reverse(d):
    
    return dict([(v,k) for (k,v) in d.items()])

id2event = dict_reverse(event2id)

In [4]:
id2event

{0: 'T001_21.5',
 1: 'AC01_1',
 2: 'T002_20.5',
 3: 'T003_20.5',
 4: 'T002_20',
 5: 'T001_21',
 6: 'T001_20.5',
 7: 'T003_20',
 8: 'T001_20',
 9: 'AC01_0',
 10: 'T002_19.5',
 11: 'T001_19.5',
 12: 'M024_1',
 13: 'L002_1',
 14: 'M025_1',
 15: 'M021_1',
 16: 'M025_0',
 17: 'L010_1',
 18: 'M021_0',
 19: 'M019_1',
 20: 'M024_0',
 21: 'M018_1',
 22: 'M017_1',
 23: 'M018_0',
 24: 'M019_0',
 25: 'M016_1',
 26: 'M015_1',
 27: 'L001_1',
 28: 'M017_0',
 29: 'L008_1',
 30: 'M014_1',
 31: 'M016_0',
 32: 'M014_0',
 33: 'M008_1',
 34: 'M009_1',
 35: 'M015_0',
 36: 'M008_0',
 37: 'M009_0',
 38: 'L001_0',
 39: 'M007_1',
 40: 'M006_1',
 41: 'M005_1',
 42: 'M007_0',
 43: 'M006_0',
 44: 'M005_0',
 45: 'T002_22',
 46: 'T002_23',
 47: 'T003_19.5',
 48: 'T002_23.5',
 49: 'T002_24',
 50: 'T002_24.5',
 51: 'T002_25',
 52: 'M011_1',
 53: 'M011_0',
 54: 'M010_1',
 55: 'M010_0',
 56: 'T002_25.5',
 57: 'M001_1',
 58: 'M023_1',
 59: 'M022_1',
 60: 'M023_0',
 61: 'M001_0',
 62: 'M022_0',
 63: 'L003_1',
 64: 'M026_1

In [5]:
# 读取所有设备
col_names = [ 'date','time', 'sensor_id', 'val']

homeass_df = pd.read_csv('event_result.log', sep="\s+", names=col_names)

homeass_df.head()

,date,time,sensor_id,val
0,2007-10-25,15:04:24.590379,T001,21.5
1,2007-10-25,15:04:24.595221,AC01,ON
2,2007-10-25,15:04:24.590379,T002,20.5
3,2007-10-25,15:04:24.590379,T003,20.5
4,2007-10-25,15:13:22.086372,T002,20


In [6]:
# 这里，直接调用两两事转移概率的矩阵
event_trans = np.load("event_trans_pro.npy")

In [7]:
homeass_df.sensor_id.unique()

array(['T001', 'AC01', 'T002', 'T003', 'M024', 'L002', 'M025', 'M021',
       'L010', 'M019', 'M018', 'M017', 'M016', 'M015', 'L001', 'L008',
       'M014', 'M008', 'M009', 'M007', 'M006', 'M005', 'M011', 'M010',
       'M001', 'M023', 'M022', 'L003', 'M026', 'M027', 'M028', 'M029',
       'M037', 'L005', 'L007', 'M030', 'DOOR01', 'M036', 'M032', 'M033',
       'M031', 'M013', 'M035', 'M034', 'F002', 'L006', 'M002', 'M003',
       'L009', 'M004', 'M012', 'L004', 'F001', 'D007', 'I003', 'I007',
       'I009', 'I008', 'A001', 'A002', 'L011', 'A003', 'I002', 'I004',
       'I005', 'I006', 'I001', 'M038', 'M039', 'M040', 'M041', 'E001'],
      dtype=object)

In [8]:
# 以下几个部分是对数据的格式化处理， 和statistic_by_state相同
homeass_df = homeass_df.applymap(lambda x: '1' if (x=='on' or x=='OPEN') else x)
homeass_df = homeass_df.applymap(lambda x: '0' if (x=='off' or x=='CLOSE') else x)

homeass_df = homeass_df.applymap(lambda x: '1' if (x=='ON' or x=='open') else x)
homeass_df = homeass_df.applymap(lambda x: '0' if (x=='OFF' or x=='close') else x)

In [9]:
temp_date_time = homeass_df['date'] + " " +  homeass_df['time']

for i in tqdm(range (temp_date_time.shape[0])):
    if len(temp_date_time.iloc[i].split(':')[-1].split('.')[0]) > 2:
        size_1 = temp_date_time.iloc[i].rfind(':')
        size_2 = temp_date_time.iloc[i].find('.')
        temp_date_time.iloc[i] = temp_date_time.iloc[i][0:size_1] + temp_date_time.iloc[i][size_1:size_2-1] + temp_date_time.iloc[i][size_2:-1]
        print( temp_date_time.iloc[i])

  3%|▎         | 12759/380447 [00:00<00:05, 64783.59it/s]

2007-10-26 15:17:51.66810
2007-10-26 18:53:19.37724
2007-10-26 19:09:59.22672
2007-10-27 09:10:33.09723
2007-10-30 17:45:56.06195
2007-10-30 18:23:11.93685
2007-10-30 20:23:00.83296
2007-10-31 22:35:59.48306
2007-11-01 17:41:11.84523
2007-11-01 18:59:12.81183
2007-11-01 20:08:29.90015
2007-11-01 20:34:15.40856


  6%|▌         | 23432/380447 [00:00<00:06, 58643.79it/s]

2007-11-02 17:01:40.15451
2007-11-02 18:50:11.70982
2007-11-02 22:59:28.73522
2007-11-03 15:51:53.64143
2007-11-03 17:00:03.65539
2007-11-03 17:28:16.07391
2007-11-03 19:27:13.38309
2007-11-03 19:37:49.56079
2007-11-03 19:47:37.89996
2007-11-03 20:15:11.95350
2007-11-03 21:04:33.97130
2007-11-03 21:32:21.9828


  9%|▉         | 35606/380447 [00:00<00:06, 54578.94it/s]

...

2008-04-12 18:00:58.44515
2008-04-12 18:19:39.06471
2008-04-12 18:22:15.17388
2008-04-12 18:49:31.3357
2008-04-12 18:52:25.93154
2008-04-12 19:14:05.94049
2008-04-12 19:19:25.40992
2008-04-12 19:25:38.9798
2008-04-12 19:47:20.88733
2008-04-13 08:24:04.24937
2008-04-13 08:41:44.32693
2008-04-14 09:31:29.79627
2008-04-15 15:25:06.90917
2008-04-15 16:07:24.32511
2008-04-15 17:06:05.98156
2008-04-15 17:09:40.3224


 97%|█████████▋| 370923/380447 [00:07<00:00, 46040.36it/s]

2008-04-15 17:20:40.96580
2008-04-15 22:35:05.86227
2008-04-17 09:41:09.49178
2008-04-17 09:55:37.48016
2008-04-18 15:05:42.34235
2008-04-25 21:11:26.80208
2008-04-30 17:19:35.79723
2008-05-08 09:39:57.43266
2008-05-12 21:22:48.81165
2008-05-13 19:31:39.16483


100%|██████████| 380447/380447 [00:07<00:00, 51857.07it/s]

2008-05-13 19:38:13.43846
2008-05-13 20:05:09.720
2008-05-14 20:40:36.74020


In [10]:
homeass_df['datetime'] = pd.to_datetime(temp_date_time )

del homeass_df['date']
del homeass_df['time']

In [11]:
homeass_df.head()

,sensor_id,val,datetime
0,T001,21.5,2007-10-25 15:04:24.590379
1,AC01,1,2007-10-25 15:04:24.595221
2,T002,20.5,2007-10-25 15:04:24.590379
3,T003,20.5,2007-10-25 15:04:24.590379
4,T002,20,2007-10-25 15:13:22.086372


In [12]:
# 通过稳定排序，使得事件按时间顺序排列
homeass_df.sort_values(by = ['datetime'] , ascending=[True], kind='mergesort', inplace=True)

In [13]:
all_sensor_id_list = homeass_df.sensor_id.unique().tolist()

In [14]:
# 这部分的flag 的作用在于区分正常事件和我们新加入的异常事件
homeass_df['flag'] = 0

In [15]:
# 对于入室抢劫当中所经历的传感器元件，我们需要创建新的异常事件，即依次改变他们的状态
target_list_new  =  ['M012', 'M011' , 'M010', 'M007', 'M001', 'M023',
               'M022', 'M021', 'M024', 'DOOR01', 'L001']

In [16]:
# '2008-01-11 22:00:00.000000' 是入室抢劫发生的时间点
att_seg_df = homeass_df[ (homeass_df['datetime'] >= '2008-01-11 22:00:00.000000') &
           (homeass_df['datetime'] <= '2008-01-11 23:00:00.000000')]

In [17]:
# 用于异常检测测试的序列
test_seg_df = homeass_df[ (homeass_df['datetime'] >= '2008-01-11 00:00:00.000000') &
           (homeass_df['datetime'] <= '2008-01-13 00:00:00.000000')]

In [18]:
# 这部分则是在入室抢劫异常事件发生前所有的事件序列
before_df = homeass_df[ (homeass_df['datetime'] < '2008-01-11 22:00:00.000000') ]


In [19]:
att_seg_df = att_seg_df.applymap(lambda x: 1 if (x=='1' or x=='OPEN') else x)
att_seg_df = att_seg_df.applymap(lambda x: 0 if (x=='0' or x=='CLOSE') else x)
test_seg_df = test_seg_df.applymap(lambda x: 1 if (x=='1' or x=='OPEN') else x)
test_seg_df = test_seg_df.applymap(lambda x: 0 if (x=='0' or x=='CLOSE') else x)
before_df = before_df.applymap(lambda x: 1 if (x=='1' or x=='OPEN') else x)
before_df = before_df.applymap(lambda x: 0 if (x=='0' or x=='CLOSE') else x)

In [20]:
# 根据输入的函数，来得到每一个设备在抢劫之前的最终状态， 运行过程大概不到1分钟
def get_origal_state(target_df, target_list):
    
    #  给出原件元件列表，对制定的元件状态进行改变，即入侵之。
    nrow, ncol = target_df.shape
    dict_state_val = {}
    
    for i in tqdm(range(nrow)):
        
        if target_df.iloc[i].sensor_id in set(target_list_new):
            
            target_s = target_df.iloc[i].copy()            
            dict_state_val[target_df.iloc[i].sensor_id] = target_s.val

            
    return dict_state_val

In [21]:
dict_state_val = get_origal_state(before_df, target_list_new)

100%|██████████| 138689/138689 [00:54<00:00, 2562.50it/s]


In [22]:
dict_state_val

{'M024': 0,
 'M021': 0,
 'L001': 0,
 'M007': 0,
 'M011': 0,
 'M010': 0,
 'M001': 0,
 'M023': 0,
 'M022': 0,
 'DOOR01': 0,
 'M012': 0}

In [23]:
# 入侵者触发设备的时间间隔，我们设置为5秒
time_span = dt.timedelta(seconds=5)

In [24]:
# 这是生成异常序列的实现函数

def attack_new(target_df, target_list ,dict_state_val):
    
    #  给出原件元件列表，对制定的元件状态进行改变，即入侵之。
    nrow, ncol = target_df.shape
    dict_event_score = {}
    attacked_df = pd.DataFrame(columns = ['datetime', 'sensor_id', 'val', 'flag'])
    
    start_time =     target_df.iloc[0].datetime
    temp_time = start_time
    
    for sensor_name in tqdm(target_list_new):
        
        temp_state = dict_state_val[sensor_name]
        one_a_df = target_df.iloc[0:1].copy()
        one_a_df.sensor_id = sensor_name
        one_a_df.val = 1 - temp_state
        one_a_df.datetime = temp_time
        temp_time += time_span
        attacked_df = pd.concat([attacked_df, one_a_df], axis=0)
        dict_state_val[sensor_name] = one_a_df.val
        
 
                    
    return attacked_df

In [25]:
attacked_df = attack_new(att_seg_df, target_list_new, dict_state_val)

  0%|          | 0/11 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

100%|██████████| 11/11 [00:00<00:00, 138.28it/s]


In [26]:
# 异常事件进行标注、以便于检测
attacked_df.flag = 1

In [27]:
attacked_df

,datetime,flag,sensor_id,val
138689,2008-01-11 22:00:26.566305,1,M012,1
138689,2008-01-11 22:00:31.566305,1,M011,1
138689,2008-01-11 22:00:36.566305,1,M010,1
138689,2008-01-11 22:00:41.566305,1,M007,1
138689,2008-01-11 22:00:46.566305,1,M001,1
138689,2008-01-11 22:00:51.566305,1,M023,1
138689,2008-01-11 22:00:56.566305,1,M022,1
138689,2008-01-11 22:01:01.566305,1,M021,1
138689,2008-01-11 22:01:06.566305,1,M024,1
138689,2008-01-11 22:01:11.566305,1,DOOR01,1


In [28]:
# 然后，合并正常序列和入室抢劫所产生的异常事件， 并按照时间顺序排列，最终保存到本地的csv文件当中，便于调用
new_df = pd.concat([test_seg_df, attacked_df])
new_df.sort_values(by = ['datetime'] , ascending=[True],  kind='mergesort', inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [128]:
new_df.to_csv('attacked_rub_house_from_window_new.csv', index=False)

In [29]:
new_df = pd.read_csv('attacked_rub_house_from_window_new.csv')

In [30]:
# 需要对时间戳进行一个处理
new_df.datetime = pd.to_datetime(new_df.datetime)

In [31]:
# 按照1个小时的时间间隔，对事件的序列进行分割，并对分割后的每一个子序列进行检测
time_span = dt.timedelta(seconds=3600)

def get_input_events_by_hour(whole_df):
    
    input_df_list = []
    
    nrow, ncol = whole_df.shape
    
    start_time = whole_df.iloc[0].datetime
    temp_time = start_time
    k = 0
    
    
    for i in tqdm(range(nrow)):
        
        temp_time = whole_df.iloc[i].datetime
        
        if (start_time + time_span ) <= temp_time:
            
            df_input = whole_df[k:i]
            input_df_list.append(df_input)
            k = i
            start_time = temp_time
            
    df_input = whole_df[k:]
    input_df_list.append(df_input)
    
    
    return input_df_list

In [32]:
input_df_list = get_input_events_by_hour(new_df)


100%|██████████| 7648/7648 [00:01<00:00, 4720.64it/s]


In [33]:
# 可以看到经过分割的序列共有16个子序列
# 且全部打劫生成的异常事件均在 第十一个子序列： input_df_list[10]当中
len(input_df_list)

16

In [34]:
input_df_list[10][input_df_list[10].flag == 1]

,datetime,flag,sensor_id,val
6273,2008-01-11 22:00:26.566305,1,M012,1.0
6279,2008-01-11 22:00:31.566305,1,M011,1.0
6282,2008-01-11 22:00:36.566305,1,M010,1.0
6283,2008-01-11 22:00:41.566305,1,M007,1.0
6284,2008-01-11 22:00:46.566305,1,M001,1.0
6289,2008-01-11 22:00:51.566305,1,M023,1.0
6290,2008-01-11 22:00:56.566305,1,M022,1.0
6291,2008-01-11 22:01:01.566305,1,M021,1.0
6292,2008-01-11 22:01:06.566305,1,M024,1.0
6293,2008-01-11 22:01:11.566305,1,DOOR01,1.0


In [35]:
input_df_list[9][input_df_list[9].flag == 1]

,datetime,flag,sensor_id,val


In [36]:
input_df_list[11][input_df_list[11].flag == 1]

,datetime,flag,sensor_id,val
